In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_style("whitegrid")
sns.set_context("notebook")
sns.set_palette("muted")

In [ ]:
DATA_PATH = Path("../data/processed/double_pendulum_processed.csv")
PLOT_DIR = Path("../plots")
RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

RESULTS_PATH = RESULTS_DIR / "energy_stats.csv"

In [ ]:
if not DATA_PATH.exists():
    raise FileNotFoundError(f"❌ Processed data not found at: {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
df.head()

In [ ]:
g = 9.81
L1 = 1.0
L2 = 1.0
m1 = 1.0
m2 = 1.0

In [ ]:
theta1 = df["theta1"].values
theta2 = df["theta2"].values
omega1 = df["omega1"].values
omega2 = df["omega2"].values

In [ ]:
x1 = L1 * np.sin(theta1)
y1 = -L1 * np.cos(theta1)
x2 = x1 + L2 * np.sin(theta2)
y2 = y1 - L2 * np.cos(theta2)

vx1 = L1 * omega1 * np.cos(theta1)
vy1 = L1 * omega1 * np.sin(theta1)
vx2 = vx1 + L2 * omega2 * np.cos(theta2)
vy2 = vy1 + L2 * omega2 * np.sin(theta2)

In [ ]:
KE1 = 0.5 * m1 * (vx1**2 + vy1**2)
KE2 = 0.5 * m2 * (vx2**2 + vy2**2)
KE_total = KE1 + KE2

PE1 = m1 * g * (y1 + L1 + L2)
PE2 = m2 * g * (y2 + L1 + L2)
PE_total = PE1 + PE2

TME = KE_total + PE_total

In [ ]:
df["KE"] = KE_total
df["PE"] = PE_total
df["TME"] = TME

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
t = df["time"]

sns.lineplot(x=t, y=df["KE"], label="Kinetic Energy", ax=ax)
sns.lineplot(x=t, y=df["PE"], label="Potential Energy", ax=ax)
sns.lineplot(x=t, y=df["TME"], label="Total Mechanical Energy", ax=ax)

ax.set_title("Energy Conservation Over Time")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Energy (Joules)")
plt.tight_layout()

energy_plot = PLOT_DIR / "energy_conservation.png"
plt.savefig(energy_plot, dpi=300)
plt.show()

print(f"✅ Saved plot: {energy_plot}")

In [ ]:
energy_stats = {
    "KE_mean": np.mean(KE_total),
    "PE_mean": np.mean(PE_total),
    "TME_mean": np.mean(TME),
    "TME_std": np.std(TME)
}

pd.DataFrame([energy_stats]).to_csv(RESULTS_PATH, index=False)
print(f"✅ Saved stats: {RESULTS_PATH}")